# Troubleshooting the network aggregations

Sam Maurer, July 2018 - Python 3.6

In [1]:
import orca
import pandana as pdna
import pandas as pd
from urbansim.utils import networks
import os; os.chdir('..')
import warnings;warnings.simplefilter('ignore')

### Look at the network

In [2]:
d = 'data/'
nodessmall = pd.read_csv(d + 'bay_area_tertiary_strongly_nodes.csv')
edgessmall = pd.read_csv(d + 'bay_area_tertiary_strongly_edges.csv')

In [3]:
print(len(nodessmall))
nodessmall.head(3)

30826


,osmid,x,y,ref,highway
0,281266,-122.302578,37.560184,414B,motorway_junction
1,302878,-122.059974,37.348484,NaN,NaN
2,302883,-122.059750,37.337687,19A,motorway_junction


In [4]:
print(len(edgessmall))
edgessmall.head(3)

65524


,uniqueid,u,v,key,oneway,highway,name,length,lanes,width,est_width,maxspeed,access,service,bridge,tunnel,area,junction,osmid,ref
0,0,309330024,253297204,0,True,motorway,Bayshore Freeway,320.849,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256787725,US 101
1,1,247385811,247385812,0,True,primary,Wolfe Road,395.862,"['4', '3']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[158248728, 393494148, 413296620]",NaN
2,2,247385811,65553953,0,False,secondary,East Homestead Road,363.617,5,NaN,NaN,35 mph,NaN,NaN,NaN,NaN,NaN,NaN,"[415833672, 506531251, 415833636, 415833670]",NaN


In [5]:
nodessmall[['osmid']].describe()

,osmid
count,3.082600e+04
mean,1.092442e+09
std,1.536415e+09
min,2.812660e+05
25%,6.539825e+07
50%,2.816185e+08
75%,1.664390e+09
max,5.461053e+09


In [6]:
edgessmall[['u','v','length']].describe()

,u,v,length
count,6.552400e+04,6.552400e+04,65524.000000
mean,1.008384e+09,1.002995e+09,479.966123
std,1.505452e+09,1.499072e+09,1123.813497
min,2.812660e+05,2.812660e+05,0.208000
25%,6.533394e+07,6.533409e+07,48.300750
50%,2.576782e+08,2.576985e+08,173.583000
75%,1.241512e+09,1.230249e+09,482.175000
max,5.461053e+09,5.461053e+09,30915.501000


### Look at parcels

In [7]:
# Code from datasources.py

@orca.table(cache=True)
def parcels():
#    df = pd.read_hdf(d + 'bayarea_ual.h5', 'parcels')
    df = pd.read_csv(d + 'mtc_data_platform_format_7-6-18/' + 'parcel_attr.csv')
    return df

@orca.table(cache=True)
def buildings():
#    df = pd.read_hdf(d + 'bayarea_ual.h5', 'buildings')
    df = pd.read_csv(d + 'mtc_data_platform_format_7-6-18/' + 'buildings_v2.csv')
    return df

orca.broadcast('parcels', 'buildings', cast_index=True, onto_on='parcel_id')

In [8]:
p = orca.get_table('parcels').to_frame()

In [9]:
print(len(p))
p.head(3)

1956207


,primary_id,development_type_id,land_value,acres,county_id,zone_id,proportion_undevelopable,tax_exempt_status,apn,parcel_id_local,geom_id,imputation_flag,x,y,shape_area,block_id,node_id
0,229116,1,0.0,3.360520,1,725,0.0,0,099 029001700,,10305106092872,_,-121.795620,37.655379,13599.555578,60014511022061,53091636
1,244166,1,0.0,1.294423,1,715,0.0,0,099B540210200,,11107351665227,_,-121.713004,37.717277,5238.347696,60014511011025,53059159
2,202378,25,6036500.0,14.993605,1,820,0.0,1,085A643106000,,11030175960628,merged,-122.014199,37.655260,60677.019451,60014351031029,1852532916


### Load network into pandana

In [10]:
# Code from Paul in Slack

@orca.injectable('netsmall', cache=True)
def build_networksmall():
    nodessmall = pd.read_csv(d + 'bay_area_tertiary_strongly_nodes.csv').set_index('osmid')
    edgessmall = pd.read_csv(d + 'bay_area_tertiary_strongly_edges.csv')
    netsmall = pdna.Network(nodessmall.x, nodessmall.y, edgessmall.u, \
        edgessmall.v, edgessmall[['length']], twoway=False)
    netsmall.precompute(5000)
    return netsmall

In [11]:
# Code from Paul in Slack

parcels = orca.get_table('parcels').to_frame(columns=['x','y'])
idssmall_parcel = orca.get_injectable('netsmall').get_node_ids(parcels.x, parcels.y)
orca.add_column('parcels', 'node_id_small', idssmall_parcel, cache=False)
orca.broadcast('nodessmall', 'parcels', cast_index=True, onto_on='node_id_small')

In [12]:
orca.get_table('parcels').to_frame().node_id_small.describe()

count    1.956207e+06
mean     7.947319e+08
std      1.402735e+09
min      2.812660e+05
25%      5.787896e+07
50%      6.550126e+07
75%      6.256488e+08
max      5.458527e+09
Name: node_id_small, dtype: float64

In [13]:
# This is what network_test.yaml looks like

'''
model_type: networks
node_col: node_id_small
variable_definitions:

  - name: residential_units_500
    dataframe: buildings
    varname: residential_units
    radius: 500
'''

'\nmodel_type: networks\nnode_col: node_id_small\nvariable_definitions:\n\n  - name: residential_units_500\n    dataframe: buildings\n    varname: residential_units\n    radius: 500\n'

In [14]:
# Code adapted from models.py

@orca.step()
def network_aggregations_small(netsmall):
    nodessmall = networks.from_yaml(netsmall, 'network_test.yaml')
    orca.add_table('nodessmall', nodessmall)

### Look at aggregations

In [15]:
orca.run(['network_aggregations_small'])

Running step 'network_aggregations_small'
Computing accessibility variables
Computing residential_units_500


KeyError: 'node_id_small'